In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
df_flights=pd.read_csv('/kaggle/input/dataset-of-agoda-hotels/flights.csv')
df_users=pd.read_csv('/kaggle/input/dataset-of-agoda-hotels/users.csv')
df_hotels=pd.read_csv('/kaggle/input/dataset-of-agoda-hotels/hotels.csv')
df_flights

In [ ]:
df_hotels

Observation: The dataset is relatively large so we are going to cut it down to smaller section for ease.

In [ ]:
df_users

In [ ]:
df_users.describe()

In [ ]:
df_users.info()

In [ ]:
df_users.set_index('code' , inplace=True)

In [ ]:
df_users.duplicated().sum()

In [ ]:
df_users.gender.value_counts(normalize=True)*100

In [ ]:
df_users.gender.value_counts(normalize=True)*100

plt.figure(figsize=(8, 5))
n, bins, patches = plt.hist(df_users['age'], bins=20, edgecolor='black')
cmap = plt.get_cmap('Greens')
norm = plt.Normalize(vmin=min(bins), vmax=max(bins))

for patch, bin_value in zip(patches, bins):
    color = cmap(norm(bin_value))
    patch.set_facecolor(color)
    
plt.xticks(ticks=np.arange(0, df_users['age'].max(), step=5), fontsize=12)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Count', fontsize=14)


plt.show()

In [ ]:
df_merged = pd.merge(df_users, df_hotels, left_on= 'code', right_on='userCode', how='inner')
df_merged.rename(columns={'name_x':'user_name','name_y':'hotel_name'}, inplace=True)

In [ ]:
df_merged.describe()

In [ ]:
df_merged['date'] = pd.to_datetime(df_merged['date'])
df_merged['date'] = df_merged['date'].dt.strftime('%Y-%m-%d')
df_merged

In [ ]:
df_merged.isna().sum()

In [ ]:
mean_sales = df_merged.groupby(['place'])['company'].value_counts().unstack()
plt.figure(figsize=(12,6))
mean_sales.plot(kind='bar')

plt.xlabel('Companies')
plt.ylabel('Number of sales')
plt.show()

In [ ]:
colors = plt.get_cmap('Pastel1_r').colors

df_merged.company.value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(7,5), title='Companies sales share', 
                                    shadow=True , colors=colors)
plt.show()

In [ ]:
distinct_passengers = df_merged.groupby('hotel_name')['userCode'].nunique().reset_index()
distinct_passengers = distinct_passengers.sort_values(by='userCode', ascending=False)
fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(distinct_passengers['hotel_name'], distinct_passengers['userCode'], color='skyblue')

ax.set_xlabel('Number of Distinct Passengers')
ax.set_ylabel('Place')
ax.set_title('Number of Distinct Passengers by Place')

plt.show()

In [ ]:
mean_sales = df_merged.groupby('hotel_name')['total'].sum()
mean_sales = mean_sales.sort_values(ascending=False)
mean_sales.plot(kind='bar')
plt.title('Total earnings of hotels')
plt.xlabel('Hotels')
plt.ylabel('Million dollars')
plt.show()

In [ ]:
crosstab= pd.crosstab(df_merged['gender'] , df_merged['place']) 
plt.figure(figsize=(12, 6))
sns.heatmap(crosstab, annot=True, fmt='d', cbar=True ,edgecolor='black')

plt.title('Gender vs places')
plt.xlabel('Places')
plt.ylabel('Gender')

plt.show()

**Observation**: Florianopolis appears to be the least popular among all, males prefer Salvador (BH), and women Rio de Janeiro (RJ)

In [ ]:
crosstab = pd.crosstab(df_merged['company'] , df_merged['days']) 
crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(crosstab, annot=True, fmt='d', cbar=True ,cmap="coolwarm")

plt.title('Companies and days of stay relation')
plt.xlabel('Number of days')
plt.ylabel('Companies')

plt.show()

In [ ]:
df = pd.merge(df_merged, df_flights, on= 'travelCode', how='inner')
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
corr = df.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))
plt.figure(figsize=(25, 7))
plt.imshow(corr, cmap='Blues')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)

for i in range(len(corr)):
    for j in range(len(corr)):
        if not mask[i, j]:
            plt.text(j, i, f"{corr.iloc[i, j]:.2f}", ha='center', va='center', color='black')

plt.show()

**Observation**: The longer the flight, the higher the fee.

In [ ]:
df['date_y'] = pd.to_datetime(df['date_y'])
df['date_y'] = df['date_y'].dt.strftime('%Y-%m-%d')
df.rename(columns={'date_x':'booking_date','price_x':'room_price', 
                   'price_y':'flight_price', 'date_y':'flight_date'}, inplace=True)
df

In [ ]:
df['time'].describe()

In [ ]:
df['flightType'].value_counts()
colors = plt.get_cmap('Pastel1').colors
df['flightType'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(10,5), title='Distribution of  Travel Type', shadow=True , colors=colors)

**Observation**: amazingly the majority of passengers prefer to fly in Business and Premium classes, rather than economy.

In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='age', bins=20, edgecolor='black', hue='flightType',  palette="pastel")
plt.xticks(ticks=np.arange(0, df['age'].max() + 5, step=5), fontsize=12)
plt.xlabel('Age', fontsize=14)
plt.ylabel("Frequency", fontsize=14)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
n, bins, patches = plt.hist(df['distance'], bins=20, edgecolor='black')
cmap = plt.get_cmap('Blues')
norm = plt.Normalize(vmin=min(bins), vmax=max(bins))

for patch, bin_value in zip(patches, bins):
    color = cmap(norm(bin_value))
    patch.set_facecolor(color)
    
plt.xticks(ticks=np.arange(0, df['distance'].max(), step=100), fontsize=12)
plt.xlabel("Flight Distance", fontsize=14)
plt.ylabel("Frequency", fontsize=14)


plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='flightType', y='flight_price', data=df)

plt.xlabel('Flight Type')
plt.ylabel('Flight Price')
plt.title('Flight Price by Flight Type')

plt.show()

In [ ]:
cut_series = pd.cut(df['distance'], bins=[0, 200, 400, 600, 800, 1000], 
                    labels=['0-200', '200-400', '400-600', '600-800', '800-1000'])
value_counts = cut_series.value_counts()

percentage_distribution = (value_counts / df['distance'].shape[0]).mul(100).round(2)

print("Distribution (Counts):")
print(value_counts)
print("\nDistribution (Percentage):")
print(percentage_distribution)

In [ ]:
plt.scatter(df['distance'], 
            df['flight_price'], alpha = 0.5 , 
            linewidths=0.3 ,edgecolors='black')

In [ ]:
df['total_earning'] = df['room_price']+df['flight_price']
labels = ['Flight Price', 'Room Price']
total_flight_cost = df['flight_price'].sum()
total_room_cost = df['room_price'].sum()
sizes = [total_flight_cost, total_room_cost]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=['skyblue', 'lightcoral'])
ax.axis('equal')
plt.title('Revenue Share')

plt.show()

In [ ]:
df['route'] = df['from']+' - '+df['to']
df['route'].head(10)

In [ ]:
passenger_count = df['agency'].value_counts()
top_10_destinations = passenger_count.head(10)
plt.figure(figsize=(12, 6))
top_10_destinations.plot(kind='bar', color='blue')
plt.xlabel('Agencies')
plt.ylabel('Sales Count')
plt.title('Agencies Performance')
plt.xticks(rotation=45)

plt.show()

In [ ]:
total_revenue = df.groupby('route')['flight_price'].sum()
top_10_destinations_revenue = total_revenue.sort_values(ascending=False).head(10)

plt.figure(figsize=(12, 6))
top_10_destinations_revenue.plot(kind='bar', color='skyblue')
plt.xlabel('Routes')
plt.ylabel('Total Revenue')
plt.title('Top 10 Most Lucrative Routes')

plt.xticks(rotation=45)

plt.show()

In [ ]:
df